## Тип Tuple
Позволяет хранить несколько значений внутри одного значения. Значения внутри, в отличие от списка, могут быть разных типов.

In [8]:
x :: (Int, String)
x = (23, "abc")
x

y :: (Int, String, Bool)
y = (42, "hello", False)
y

z :: ()  -- 0 значений внутри Tuple
z = ()
z

t :: (Int)  -- эквивалентно типу Int
t = (42)  

Line 13: Redundant bracket
Found:
(Int)
Why not:
IntLine 14: Redundant bracket
Found:
(42)
Why not:
42

(23,"abc")

(42,"hello",False)

()

Значение `()` иногда возвращают, когда ничего не хочется возвращать.

Как разбирать значения типа tuple на составляющие? Через сопоставление с образцом. Например, мы работаем с точками типа (Float, Float), т.е. у нас две координаты у точки. Давайте посчитаем что-то про точки:

In [9]:
distTo0 :: (Float, Float) -> Float
distTo0 (x, y) = sqrt(x*x + y*y)

distTo0 (3, 4)

5.0

функция distTo0 вызывается, на самом деле, от одного аргумента типа (Float, Float).

In [11]:
dist :: (Float, Float) -> (Float, Float) -> Float
dist (x1, y1) (x2, y2) = sqrt(dx*dx + dy*dy) where
                                               dx = x2 - x1
                                               dy = y2 - y1
                                               
dist (1, 0) (5, 0)

4.0

Есть встроенные функции для работы с Tuple: `fst` и `snd`:

In [16]:
fst' :: (a, b) -> a
fst' (x, _) = x

snd' :: (a, b) -> b
snd' (_, y) = y

swap :: (a, b) -> (b, a)
swap (x, y) = (y, x)

fst' (10, "abc")
snd' (10, "abc")
swap (10, "abc")

10

"abc"

("abc",10)

Можно использовать списки тьюплов. Для работы с ними есть ф-ия zip и unzip:

In [19]:
-- zip :: [a] -> [b] -> [(a, b)]
zip [1, 2, 3, 4] [True, False, False, True]

zip' :: [a] -> [b] -> [(a, b)]
zip' [] _ = []
zip' _ [] = []
zip' (h1:t1) (h2:t2) = (h1, h2) : zip' t1 t2

zip' [1, 2, 3, 4] [True, False, False, True]

[(1,True),(2,False),(3,False),(4,True)]

[(1,True),(2,False),(3,False),(4,True)]

In [20]:
-- unzip :: [(a, b)] -> ([a], [b])
unzip [(2, 3), (3, 4), (10, 5)]

([2,3,10],[3,4,5])

`zipWith` более общая, чем zip, она позволяет дополнительно указать, как объединять пару элементов списоков.

In [21]:
:type zipWith

zipWith :: forall a b c. (a -> b -> c) -> [a] -> [b] -> [c]

In [24]:
-- zipWith (\x y -> x + y) [1, 2, 3, 4] [10, 20, 30, 40]
zipWith (+) [1, 2, 3, 4] [10, 20, 30, 40]

[11,22,33,44]

# Частичное применение функции
Если есть какая-то функция, можно вызвать ее только с частью аргументов, тогда получится функция, которая хочет получить оставшиеся аргументы:

In [33]:
f :: Int -> Int -> Int -> Int
f x y z = 100 * x + 10 * y + z
f 1 2 3

g = f 1  -- эта функция такая g y z = f 1 y z
g 5 6
g 4 8

h = f 1 2
h 8  -- f 1 2 8
h 5  -- f 1 2 5

123

156

148

128

125

Можно считать, что в Haskell все функции на самом деле имеют ровно один аргумент:
`f` функция одного аргумента, `f 1` возвращает новую функцию `g`, которая тоже требует Int аргумента.
`g` функция оного аргумента. `g 2` — это очередная функция `h`, которая опять требует Int аргумент.
`h` функция одного аргумента. Она получает Int аргумент, и приписывает его в конец числа `12`:

In [35]:
f x y z = 100*x + 10*y + z
g = f 1
h = g 2
h 3
h 4

123

124

Получается, когда мы пишем `f 1 2 3`, мы имеем в виду `((f 1) 2) 3`.
Это же объясняет, почему тип функции записывается через `->`:

```
h :: Int -> Int
g :: Int -> (Int -> Int)
f :: Int -> (Int -> (Int -> Int))
```

In [36]:
f :: Int -> (Int -> (Int -> Int)) -- корректная запись типа
f x y z = 100*x + 10*y + z

Типовый оператор `->` правоассоциативен. Т.е. a -> b -> c - на самом деле это a -> (b -> c)

Сравните с `10 - 20 - 45` имеется в виду `(10 - 20) - 45`

Давайте еще для понимания посмотрим на похожие типы:

In [40]:
fun :: Int -> (Int -> Int) -- или просто Int -> Int -> Int
fun x y = x + 3 * y

fun2 :: (Int -> Int) -> Int -- это совсем не то же самое Int -> Int -> Int
fun2 op = op 0

fun2 (+1)
fun2 (\x -> x * 10 + 5)
fun2 (2^) -- (\x -> 2 ^ x)

1

5

1

Section `(+2)`, `(/2)`, `(2/)` - это частный случай частичного применения функции. В section можно указать и первый, и второй аргумент функции. В примерах выше мы могли указать только первый аргумент:
`f 1`

In [48]:
f :: Int -> Int -> Int -> Int
f x y z = 100*x + 10*y + z
g = f 1 -- g = \y z -> f 1 y z
-- g = \x z -> f x 2 y это частичное применение второй аргумент

Второй вариант сложнее записать коротко. Но можно:

In [54]:
f 1 2 3
-- `f` позволяет пользоваться функцией инфиксно
(4 `f` 5) 6
g = (`f` 2) -- позволило мне сделать частичное применение f,
            -- указав только второй аргумент
g 4 6

123

456

426

Полезные примеры частичного применения:

In [56]:
-- map :: (a -> b) -> [a] -> [b]
inc1 = map (+1)
inc1 [1, 2, 3]
inc1 [10, 20, 30]

[2,3,4]

[11,21,31]

Как бы мы записали функцию `inc1`, не зная про частичное применение:

In [ ]:
inc1 :: [Int] -> [Int]
inc1 list = map (+1) list -- сравним с inc1 = map (+1)

Вывод, при определении функции можно сокращать на одинаковый аргумент в конце.

Пример:

In [57]:
filterOdds :: [Int] -> [Int]
filterOdds list = filter (\x -> x `mod` 2 == 1) list --можно mod x 2

filterOdds [1, 2, 3, 4, 5]

Line 2: Eta reduce
Found:
filterOdds list = filter (\ x -> x `mod` 2 == 1) list
Why not:
filterOdds = filter (\ x -> x `mod` 2 == 1)

[1,3,5]

Предалгается убрать list в конце:

In [58]:
filterOdds :: [Int] -> [Int]
filterOdds = filter (\x -> x `mod` 2 == 1) --можно mod x 2

filterOdds [1, 2, 3, 4, 5]

[1,3,5]

In [59]:
filterOdds = filter (\x -> x `mod` 2 == 1)

In [60]:
:type filterOdds

filterOdds :: forall a. Integral a => [a] -> [a]